<h1 align="center">Computación de Alto Desempeño</h1>
<h1 align="center">OpenMP</h1>
<h1 align="center">Ejemplo 01: Cálculo de pi por integración numérica</h1>
<h1 align="center">2024</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

***
|[![Outlook](https://img.shields.io/badge/Microsoft_Outlook-0078D4?style=plastic&logo=microsoft-outlook&logoColor=white)](mailto:calvarezh@udemedellin.edu.co)||[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlosalvarezh/HPC/blob/main/HPC08_Ej01_CalculoPI.ipynb)
|-:|:-|--:|
|[![LinkedIn](https://img.shields.io/badge/linkedin-%230077B5.svg?style=plastic&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/carlosalvarez5/)|[![@alvarezhenao](https://img.shields.io/twitter/url/https/twitter.com/alvarezhenao.svg?style=social&label=Follow%20%40alvarezhenao)](https://twitter.com/alvarezhenao)|[![@carlosalvarezh](https://img.shields.io/badge/github-%23121011.svg?style=plastic&logo=github&logoColor=white)](https://github.com/carlosalvarezh)|

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/Curso_CEC_EAFIT/blob/main/images/CCLogoColorPop1.gif?raw=true" width="25">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

***

## Cálculo de $\pi$ mediante integración numérica

Calcular el valor de $\pi$ mediante integración numérica, utilizando la fórmula:

$$
\pi = \int_0^1 \frac{4}{1 + x^2} \, dx
$$

Para aproximar la integral, utilizamos la **suma de rectángulos**:

$$
\sum_{i=0}^{N-1} F(x_i) \cdot \Delta x \approx \pi
$$

donde:
- $x_i$ es el punto en el intervalo en el que se evalúa la función.
- $F(x_i) = \frac{4}{1 + x_i^2}$ es el valor de la función en el punto $x_i$.
- $\Delta x = \frac{1}{N}$ es el ancho de cada rectángulo, donde $N$ es el número de rectángulos que usamos para aproximar la integral.



### Código Serial

Primero, implementaremos el cálculo de $\pi$ utilizando un enfoque secuencial en C.

In [ ]:
#include <stdio.h>

int main() {
    long long N = 1000000;  // Número de intervalos (mayor N, más precisión)
    double pi = 0.0;
    double delta_x = 1.0 / (double)N;  // Ancho de cada rectángulo

    // Bucle para sumar las áreas de los rectángulos
    for (long long i = 0; i < N; i++) {
        double x_i = (i + 0.5) * delta_x;  // Punto medio del intervalo
        pi += 4.0 / (1.0 + x_i * x_i);     // Añadir altura del rectángulo a la suma
    }

    pi *= delta_x;  // Multiplicar por el ancho del rectángulo para obtener el área total

    printf("Valor aproximado de pi: %.15f\n", pi);
    return 0;
}

**Explicación del Código Serial:**

1. **Inicialización de Variables:**
   - `N`: Número de intervalos o rectángulos para la integración.
   - `pi`: Variable que almacenará el valor aproximado de $\pi$.
   - `delta_x`: Ancho de cada rectángulo, dado por $\Delta x = \frac{1}{N}$.

2. **Ciclo de Suma:**
   - En el ciclo `for`, se suman las alturas de los rectángulos en cada punto medio $x_i$, que se calcula como $x_i = (i + 0.5) \cdot \Delta x$.
   - La función que determina la altura de cada rectángulo es $F(x_i) = \frac{4}{1 + x_i^2}$, que se suma a la variable `pi`.

3. **Multiplicación Final:**
   - Al final, multiplicamos el valor de `pi` acumulado por $\Delta x$ para obtener el área total bajo la curva, que es la aproximación de $\pi$.

**Salida Esperada del Código Serial:**

```
Valor aproximado de pi: 3.141592653589793
```

### Paralelización del Código con OpenMP

Ahora que tenemos la versión secuencial, procedemos a analizar cómo podemos paralelizar el cálculo con OpenMP. Para ello, identificamos las partes del código que pueden ejecutarse en paralelo.

**Análisis para la Paralelización:**

1. **Identificación de la Parte Paralelizante:**
   - El bucle `for` es independiente para cada iteración, lo que significa que no hay dependencia entre las iteraciones. Cada iteración solo calcula un valor para un rectángulo específico y lo añade a la suma de `pi`.

2. **Riesgo de Condiciones de Carrera:**
   - Si varios hilos intentan actualizar la misma variable `pi` al mismo tiempo, pueden ocurrir **condiciones de carrera**. Para evitarlo, utilizaremos la cláusula `reduction` de OpenMP para asegurar que cada hilo trabaje en su propia copia de la variable `pi` y combine los resultados al final.

**Código Paralelo con OpenMP:**

In [ ]:
#include <omp.h>
#include <stdio.h>

int main() {
    long long N = 1000000;  // Número de intervalos (mayor N, más precisión)
    double pi = 0.0;
    double delta_x = 1.0 / (double)N;  // Ancho de cada rectángulo

    // Paralelizar la suma utilizando OpenMP
    #pragma omp parallel
    {
        double sum_local = 0.0;  // Variable local para cada hilo
        #pragma omp for
        for (long long i = 0; i < N; i++) {
            double x_i = (i + 0.5) * delta_x;  // Punto medio del intervalo
            sum_local += 4.0 / (1.0 + x_i * x_i);
        }

        // Combinar resultados parciales en la variable global pi
        #pragma omp atomic
        pi += sum_local * delta_x;
    }

    printf("Valor aproximado de pi: %.15f\n", pi);
    return 0;
}

**Explicación de la Versión Paralela:**

1. **Región Paralela:**
   - Usamos `#pragma omp parallel` para iniciar una región paralela donde cada hilo calcula una parte de la suma total.

2. **Variable Local `sum_local`:**
   - Cada hilo tiene su propia copia de `sum_local` para evitar condiciones de carrera. Cada hilo suma sus valores en `sum_local` en el bucle paralelo.

3. **Combinar Resultados:**
   - Una vez que cada hilo ha calculado su parte, utilizamos `#pragma omp atomic` para asegurar que la actualización de la variable global `pi` se realice de forma segura (es decir, sin interferencia entre los hilos).


### Métricas Clave para Evaluar el Desempeño:


Para determinar el **desempeño** de las implementaciones secuencial y paralela del cálculo de $\pi$, puedes usar varias **métricas de rendimiento** que ayudarán a comparar cuál de las dos es más eficiente. Aunque ambas implementaciones deberían dar el mismo resultado final, el enfoque de paralelización está diseñado para reducir el tiempo de ejecución, especialmente en sistemas con múltiples núcleos.

1. **Tiempo de Ejecución** (Execution Time):
   - **Descripción**: Es el tiempo total que tarda cada implementación (secundaria y paralela) en completar el cálculo de $\pi$.
   - **Cómo medirlo**: Puedes usar temporizadores como `omp_get_wtime()` en OpenMP o funciones de la biblioteca estándar como `clock()` o `chrono` en C++.
   - **Comparación**: El tiempo de ejecución de la versión paralela debería ser menor que el de la versión secuencial si la paralelización está bien implementada y el sistema tiene suficientes recursos (como núcleos).

   **Ejemplo en C usando `omp_get_wtime()`**:
   ```c
   double start_time = omp_get_wtime();
   // Código a medir
   double end_time = omp_get_wtime();
   printf("Tiempo de ejecución: %f segundos\n", end_time - start_time);
   ```

2. **Aceleración** (Speedup):
   - **Descripción**: Es una métrica que mide cuánto más rápido es el código paralelo en comparación con el código secuencial. Se calcula como la razón entre el tiempo de ejecución del código secuencial y el tiempo de ejecución del código paralelo.
   - **Fórmula**: 
     $$
     \text{Speedup} = \frac{T_{\text{sec}}}{T_{\text{par}}}
     $$
     donde $T_{\text{sec}}$ es el tiempo de ejecución del código secuencial y $T_{\text{par}}$ es el tiempo de ejecución del código paralelo.
   - **Interpretación**: Un valor de **Speedup** mayor que 1 indica que la paralelización mejoró el rendimiento. Si el **Speedup** es 1, significa que la paralelización no tuvo ningún efecto, y si es menor que 1, indica que la paralelización empeoró el rendimiento (algo no está bien implementado).

3. **Eficiencia** (Efficiency):
   - **Descripción**: Es la proporción de la aceleración obtenida en comparación con el número de hilos utilizados. Evalúa si los recursos paralelos (como núcleos) se están utilizando de manera efectiva.
   - **Fórmula**: 
     $$
     \text{Eficiencia} = \frac{\text{Speedup}}{P}
     $$
     donde $P$ es el número de hilos o procesadores utilizados.
   - **Interpretación**: Una eficiencia cercana a 1 indica un buen uso de los recursos paralelos. A medida que aumentas el número de hilos, la eficiencia tiende a disminuir debido a la sobrecarga de coordinación entre hilos.

4. **Uso de CPU** (CPU Utilization):
   - **Descripción**: Mide el porcentaje de tiempo que el procesador está ocupado durante la ejecución del programa. En la versión paralela, deberías observar un uso elevado de la CPU cuando todos los núcleos están trabajando.
   - **Cómo medirlo**: En sistemas UNIX/Linux, herramientas como `top` o `htop` pueden mostrar el uso de la CPU. En Windows, puedes usar el "Administrador de tareas".

5. **Escalabilidad**:
   - **Descripción**: Evalúa cómo cambia el rendimiento al variar el número de hilos o procesadores. Una buena implementación paralela debería escalar bien, es decir, debería mejorar el rendimiento al aumentar el número de hilos.
   - **Cómo medirla**: Ejecuta la versión paralela con diferentes números de hilos (por ejemplo, 1, 2, 4, 8, 16) y obseva cómo varía el tiempo de ejecución.



Para medir el **tiempo de ejecución**, tanto en la versión secuencial como en la paralela, puedes usar `omp_get_wtime()` en OpenMP. 

### **Versión Secuencial:**


In [ ]:
#include <stdio.h>
#include <omp.h>

int main() {
    long long N = 1000000;
    double pi = 0.0;
    double delta_x = 1.0 / (double)N;

    // Iniciar el temporizador
    double start_time = omp_get_wtime();

    for (long long i = 0; i < N; i++) {
        double x_i = (i + 0.5) * delta_x;
        pi += 4.0 / (1.0 + x_i * x_i);
    }

    pi *= delta_x;

    // Detener el temporizador
    double end_time = omp_get_wtime();

    printf("Valor aproximado de pi: %.15f\n", pi);
    printf("Tiempo de ejecución (secuencial): %f segundos\n", end_time - start_time;

    return 0;
}

### **Versión Paralela:**


In [ ]:
#include <omp.h>
#include <stdio.h>

int main() {
    long long N = 1000000;
    double pi = 0.0;
    double delta_x = 1.0 / (double)N;

    // Iniciar el temporizador
    double start_time = omp_get_wtime();

    #pragma omp parallel
    {
        double sum_local = 0.0;
        #pragma omp for
        for (long long i = 0; i < N; i++) {
            double x_i = (i + 0.5) * delta_x;
            sum_local += 4.0 / (1.0 + x_i * x_i);
        }

        #pragma omp atomic
        pi += sum_local * delta_x;
    }

    // Detener el temporizador
    double end_time = omp_get_wtime();

    printf("Valor aproximado de pi: %.15f\n", pi);
    printf("Tiempo de ejecución (paralelo): %f segundos\n", end_time - start_time);

    return 0;
}

### **Ejemplo de Análisis de Desempeño:**


1. Ejecuta ambos códigos (secuencial y paralelo).
2. Mide el **tiempo de ejecución** de cada uno.
3. Calcula el **speedup**:
   $$
   \text{Speedup} = \frac{T_{\text{sec}}}{T_{\text{par}}}
   $$
   Si, por ejemplo, el tiempo de ejecución secuencial es de 2.0 segundos y el tiempo paralelo es de 0.5 segundos con 4 hilos, el **speedup** sería:
   $$
   \text{Speedup} = \frac{2.0}{0.5} = 4.0
   $$
4. Calcula la **eficiencia**:
   $$
   \text{Eficiencia} = \frac{\text{Speedup}}{P}
   $$
   Si usamos 4 hilos, la eficiencia sería:
   $$
   \text{Eficiencia} = \frac{4.0}{4} = 1.0
   $$
   Esto indicaría una eficiencia perfecta (100%).